# 2.1 Classification Task

#### 
#### Dependencies:

In [30]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score

from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

#### 
#### Load data

In [15]:
# Train set
X_train = np.load("./classification_data/Cancer_Xtrain.npy")
y_train = np.load("./classification_data/Cancer_ytrain.npy").ravel()
print(X_train.shape)
print(y_train.shape)

# Test set
X_test = np.load("./classification_data/Cancer_Xtest.npy")
y_test = np.load("./classification_data/Cancer_ytest.npy").ravel()
print(X_test.shape)
print(y_test.shape)

(92, 9)
(92,)
(24, 9)
(24,)


#### 
#### Init and fit

In [139]:
clf = make_pipeline(StandardScaler(), SVC(C=5,gamma='auto',kernel="rbf"))
clf.fit(X_train, y_train.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, gamma='auto'))])

In [140]:
y_pred = clf.predict(X_train)

print("Accuracy:", accuracy_score(y_pred, y_train))
print("F_measure:", f1_score(y_pred, y_train))

Accuracy: 0.967391304347826
F_measure: 0.9647058823529412


#### 
#### Predict and evaluate SVM classifier

In [141]:
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_pred, y_test))
print("F_measure:", f1_score(y_pred, y_test))

Accuracy: 0.9166666666666666
F_measure: 0.8888888888888888


#### 
#### Adjust data for neural network

In [6]:
# Scale values to 0-1
scaler = StandardScaler().fit(X_train)
X_train_nn = scaler.transform(X_train)
X_test_nn = scaler.transform(X_test) 

# Change format on labels
y_train_nn = to_categorical(y_train-1)
y_test_nn = to_categorical(y_test-1)


#### 
#### Init and fit model

In [151]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=9))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.00015,clipnorm=1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 500)               5000      
_________________________________________________________________
dense_33 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_34 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 102       
Total params: 60,252
Trainable params: 60,252
Non-trainable params: 0
_________________________________________________________________


In [150]:
model.fit(X_train_nn, y_train_nn, epochs=15, batch_size=1, verbose=1,validation_data=(X_test_nn, y_test_nn))

Epoch 1/15
92/92 [==============================] - 0s 2ms/step - loss: 0.6812 - accuracy: 0.5326 - val_loss: 0.6231 - val_accuracy: 0.7917
Epoch 2/15
92/92 [==============================] - 0s 961us/step - loss: 0.6101 - accuracy: 0.7717 - val_loss: 0.5405 - val_accuracy: 0.8333
Epoch 3/15
92/92 [==============================] - 0s 912us/step - loss: 0.5704 - accuracy: 0.7826 - val_loss: 0.4790 - val_accuracy: 0.7917
Epoch 4/15
92/92 [==============================] - 0s 915us/step - loss: 0.5333 - accuracy: 0.7826 - val_loss: 0.4378 - val_accuracy: 0.8333
Epoch 5/15
92/92 [==============================] - 0s 1ms/step - loss: 0.4758 - accuracy: 0.8152 - val_loss: 0.3887 - val_accuracy: 0.8333
Epoch 6/15
92/92 [==============================] - 0s 1ms/step - loss: 0.4587 - accuracy: 0.8370 - val_loss: 0.3766 - val_accuracy: 0.8333
Epoch 7/15
92/92 [==============================] - 0s 1ms/step - loss: 0.4281 - accuracy: 0.8043 - val_loss: 0.3481 - val_accuracy: 0.8750
Epoch 8/15
92/

#### 
#### Predict and evaluate neural network

In [27]:
y_pred = np.argmax((model.predict(X_test_nn) > 0.5).astype("int32") , axis=-1) + 1
print("Accuracy:", accuracy_score(y_pred, y_test))
print("F_measure:", f1_score(y_pred, y_test))

Accuracy: 0.875
F_measure: 0.8571428571428571
